In [1]:
########################################################
# GEO419 Projekt Nr. 2
# Download of Imagery from the Thuringian Geoportal
# Marcel Felix & Friederike Metz
# from Sept. 2021 to Feb. 2022
########################################################

In [2]:
### Download URL original
## Höhendaten:
# https://geoportal.geoportal-th.de/hoehendaten/DGM/dgm_2014-2019/dgm1_682_5644_1_th_2014-2019.zip?type=dhm1&id=16541&log=682_5644_1x1km&url=%2Fhoehendaten%2FDGM%2Fdgm_2014-2019%2Fdgm1_682_5644_1_th_2014-2019.zip
## Orthophotos:
# https://geoportal.geoportal-th.de/gaialight-th/_apps/dladownload/download.php?type=op&id=255133&log=202005-32682_5644

### Gekürzt
## Höhendaten:
# DGM
# https://geoportal.geoportal-th.de/hoehendaten/DGM/dgm_2014-2019/dgm1_682_5644_1_th_2014-2019.zip
# DOM
# https://geoportal.geoportal-th.de/hoehendaten/DOM/dom_2014-2019/dom1_682_5644_1_th_2014-2019.zip
# LAS
# https://geoportal.geoportal-th.de/hoehendaten/LAS/las_2014-2019/las_682_5644_1_th_2014-2019.zip+
## Orthophotos:
# https://geoportal.geoportal-th.de/gaialight-th/_apps/dladownload/download.php?type=op&id=255133
# filename: dop20rgbi_32_641_5656_1_th_2020.zip

In [3]:
### ID Kachelbenennung
## Höhendaten: 
# 682_5644 -> 682 steht für die West-Ost-Ausrichtung (nach Osten aufsteigend)
# 682_5644 -> 5644 steht für die Nord-Süd-Ausrichtung (nach Norden aufsteigend)

# Die Kacheln (Position) für Höhendaten und Orthophotos sind identisch
# ...das bedeutet man kann identifizieren, welche Kacheln der Orthophotos gebraucht werden 
# ...ohne ihre ID vorher zu kennen. Stattdessen muss man sie berechnen:

## Orthophotos:
# 255133 -> 255 steht für ???
# 255133 -> 133 steht für ???

In [1]:
### Importblock
import os
#import gdal
import re
from osgeo import ogr,osr,gdal
import geopandas
import shapely
import zipfile
from shapely.geometry import Polygon, LineString, Point
import requests
import ipyleaflet

In [2]:

### User's choices ###############
##################################


In [ ]:
# 1. Please select your shapefile here
shapefile = "F:/Marcel/Backup/Dokumente/Studium/Geoinformatik/SoSe 2021/GEO419/Abschlussaufgabe/Data/shape/shape.shp"

In [ ]:
# 2. Please choose your dem-years (2010-2013 / 2014-2019 / 2020-2025)
dem_year = "2014-2019"

if dem_year == "2010-2013":
    request = "https://geoportal.geoportal-th.de/hoehendaten/Uebersichten/Stand_2010-2013.zip"
if dem_year == "2014-2019":
    request = "https://geoportal.geoportal-th.de/hoehendaten/Uebersichten/Stand_2014-2019.zip"
if dem_year == "2020-2025":
    request = "https://geoportal.geoportal-th.de/hoehendaten/Uebersichten/Stand_2020-2025.zip"
    # ^These are the download-links for the dem tile polygons

In [ ]:
# Automated DOWNLOAD OF TILE POLYGONS
if os.path.exists("../Data/Stand_"+dem_year) == False:
    os.mkdir("../Data/Stand_"+dem_year)

gridfile = "../Data/Stand_"+dem_year+"/Stand_"+dem_year+".zip"

if os.path.exists(gridfile) == False:
    grid_polygons = requests.get(request)
    with open(gridfile, "wb") as file:
            file.write(grid_polygons.content)

with zipfile.ZipFile(gridfile, 'r') as zip_ref:
    zip_ref.extractall("../Data/Stand_"+dem_year)
os.remove(gridfile)
            

In [ ]:
# 3. Please choose your desired file format (dgm / dom / las)
dem_format = "dgm"

In [5]:

### File choices ###############
################################


['680_5643', '680_5644', '681_5643', '681_5644']

In [ ]:
# 4. Intersection of shapefile with dem tile polygons
polygon = geopandas.read_file(shapefile)
if dem_year == "2010-2013":
    grid_path = "../Data/Stand_2010-2013/DGM2_2010-2013_Erfass-lt-Meta_UTM32-UTM_2014-12-10.shp"
elif dem_year == "2014-2019":
    grid_path = "../Data/Stand_2014-2019/DGM1_2014-2019_Erfass-lt-Meta_UTM_2020-04-20--17127.shp"
else:
    grid_path = "../Data/Stand_2020-2025/DGM1_2020-2025_Erfass-lt-Meta_UTM_2021-03--17127.shp"
    
grid = geopandas.read_file(grid_path)

tiles = []
for poly in grid.iterrows():
    geom = poly[1].geometry
    for poly2 in polygon.iterrows():
        geom2 = poly2[1].geometry
        
        if geom.overlaps(geom2) == True:
            tiles.append(poly[1].NAME)


In [ ]:
# 5. dem tile names
tiles 

In [ ]:
# 6. Identifying op tile names
def OP_Tilename_Finder(request:str, index:int):
    """Request function which returns ID and filename;
    requires the request url and an ID to query
    """
    r = requests.head(request) 
    try:
        text = r.headers["Content-disposition"]
    except KeyError: return
    
    filename = text.split("=")[1].replace('"', '')
    if "dop" in filename:
        return [index, filename]
    else: return

def Start_after_Interruption(n_jobs:int = 200, end:int = 300000):
    """If the execution has been interrupted you can launch the process from here
        This will read the last ID in the created list and continue from there, 
        to an end that you have to set"""
    # location of ipynb
    os.chdir("F:/Marcel/Backup/Dokumente/Studium/Geoinformatik/SoSe 2021/GEO419/Abschlussaufgabe/GeoportalTH_Imagery_Download/")
    
    idfile = "../Data/"+"idlist.txt"    #created list
    with open(idfile, "r") as file:
        last = int(file.readlines()[-1].split(",")[0])+1 #extracts last ID and sets it +1
    if last >= end:
        return("End already reached")
    OP_Tilelist_Creator(n_jobs, last, end)
        
def OP_Tilelist_Creator(n_jobs:int = 200, start:int = 100000, end:int = 300000):
    """Function for complete query of all tile IDs on the server within a specified range
    
    Defining query parameters in function call:
         -- n_jobs # based on empirical testing, determines the number of threads/requests 
         -- first ID
         -- last ID
         """
    # location of ipynb
    os.chdir("F:/Marcel/Backup/Dokumente/Studium/Geoinformatik/SoSe 2021/GEO419/Abschlussaufgabe/GeoportalTH_Imagery_Download/")
    
    #create file in advance
    idfile = "../Data/"+"idlist.txt" #location of file to be created
    if os.path.exists(idfile) == False:
        with open(idfile, "w") as file:
            file.write("id,filename\n")
    
    
    #requesting loop
    # end = start + n_jobs*2 # only for testing 
    for current in range(start, end, n_jobs): 
        with Parallel(n_jobs, require="sharedmem", prefer="threads") as parallel:
            names_list = parallel(delayed(OP_Tilename_Finder)("https://geoportal.geoportal-th.de/gaialight-th/_apps/dladownload/download.php?type=op&id="+str(index), index) for index in range(current, current+n_jobs))
                # will extend beyond "end" and secure that none are left out
        with open(idfile, "a+") as file:
            for entry in names_list:
                if entry != None:
                    file.write(str(entry[0]) + "," + entry[1] + "\n")
    return # names_list #for checking 

In [ ]:
# OP_Tilelist_Creator()
# only when no list available

In [ ]:
### Download request ###
########################

In [1]:
# 7. Download of dem tiles
for tile in tiles:
    # creating request url
    if dem_format == "dgm" or dem_format == "dom":
        request = "https://geoportal.geoportal-th.de/hoehendaten/"+dem_format.upper()+"/"+dem_format+"_"+dem_year+"/"+dem_format+"1_"+tile+"_1_th_"+dem_year+".zip"
    elif dem_format == "las": 
        request = "https://geoportal.geoportal-th.de/hoehendaten/"+dem_format.upper()+"/"+dem_format+"_"+dem_year+"/"+dem_format+"_"+tile+"_1_th_"+dem_year+".zip+"
    
    dem_load = requests.get(request)
    # download the file to data folder
    current_dem = "../Data/dem/"+tile+".zip"
    demzip = []
    demzip.append(current_dem)
    with open(current_dem, "wb") as file:
        file.write(dem_load.content)
        
    # unzipping
    with zipfile.ZipFile(current_dem, 'r') as zip_ref:
        zip_ref.extractall("../Data/dem/")


NameError: name 'tiles' is not defined

In [ ]:
# 8. Download of op tiles


In [5]:
### Converting XYZ to tif and Merging
# 9. Conversion & Merge

# find the xyz files
demlist = []
for file in os.scandir("../Data/dem/"):
    if file.name.endswith(".xyz"):
        demlist.append(file.path)

# convert to tif
for xyz in demlist:
    outname = xyz.replace(".xyz","")
    gdal.Translate(outname + ".tif", xyz, outputSRS="EPSG:25832")

# merging
# build virtual raster and convert to geotiff
demlist = [file.replace(".xyz",".tif") for file in demlist]
vrt = gdal.BuildVRT("../Data/dem/merged.vrt", demlist)
gdal.Translate("../Data/dem/mergedDEM2.tif", vrt, xRes = 1, yRes = -1)
   

In [ ]:

### Mosaicing with custom extent ###
####################################


In [ ]:
# 10. clipping with shapefile


In [2]:
### Visualisation ###
#####################


Help on function lol in module __main__:

lol()
    Diese funtkion macht Sachen



In [ ]:
# 10. load open street map


In [18]:
# 11. overlay mosaic